In [1]:
import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "portfolio_management.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from common.models import Brokers, Prices, Transactions, FX, Assets
from users.models import CustomUser
from utils import NAV_at_date, Irr, chart_dates, chart_labels, chart_colour, portfolio_at_date, calculate_security_nav, currency_format, calculate_portfolio_value
from datetime import date
from django.db.models import F, Sum

In [2]:
selected_brokers = broker_ids = [2]
effective_current_date = date.today()
currency_target = 'USD'
number_of_digits = 2

CustomUser.objects.get(id=2)

<CustomUser: Yaroslav>

In [3]:
security_id = 7
security = Assets.objects.get(id=security_id)
security.price_at_date(effective_current_date).price

quote = security.prices.filter(date__lte=effective_current_date).order_by('-date').first()
print(currency_target)
if currency_target is not None:
    quote.price = quote.price * FX.get_rate(security.currency, currency_target, effective_current_date)['FX']
quote

USD


<Prices: Long/short is at 22.648000000000 on 2020-12-01>

In [4]:
from django.db.models import Count

duplicates = (
            Prices.objects.values('date', 'security', 'price')
            .annotate(count_id=Count('id'))
            .filter(count_id__gt=1)
        )

total_duplicates = len(duplicates)
print((f"Found {total_duplicates} duplicate entries."))

for i, duplicate in enumerate(duplicates, start=1):
            # Keep one entry and delete the rest
            entries = Prices.objects.filter(
                date=duplicate['date'],
                security=duplicate['security'],
                price=duplicate['price']
            )
            entries_to_keep = entries.first()
            entries.exclude(id=entries_to_keep.id).delete()

            # Print status update
            print(f"Duplicates are being removed: {i} out of {total_duplicates}\r", end='', flush=True)
            
print("\nDuplicates removed.")


Found 0 duplicate entries.

Duplicates removed.


In [5]:
import datetime
from decimal import Decimal


def end_of_year_price_correction(user, year, broker_name, target_nav, asset_name):
    
    target_nav = round(Decimal(target_nav), 2)
    
    # Get the broker
    try:
        broker = Brokers.objects.get(name=broker_name)
    except Brokers.DoesNotExist:
        return {"error": f"Broker {broker_name} does not exist."}

    # Get the asset
    try:
        asset = Assets.objects.get(name=asset_name)
    except Assets.DoesNotExist:
        return {"error": f"Asset {asset_name} does not exist."}

    # Calculate end of year date
    end_of_year_date = datetime.date(year, 12, 31)

    # Fetch NAV at the end of the year
    # nav_at_end_of_year = get_nav_at_date(broker, end_of_year_date)
    nav_at_end_of_year = NAV_at_date(user.id, [broker.id], end_of_year_date, asset.currency, [])['Total NAV']
    if nav_at_end_of_year is None:
        return {"error": f"No NAV found for broker {broker_name} at the end of {year}."}

    # Fetch asset price and position at the end of the year
    price_at_end_of_year = asset.price_at_date(end_of_year_date)
    if not price_at_end_of_year:
        return {"error": f"No price found for asset {asset_name} at the end of {year}."}

    position_at_end_of_year = asset.position(end_of_year_date, [broker.id])

    # Calculate new price
    old_price = price_at_end_of_year.price
    new_price = old_price + ((target_nav - nav_at_end_of_year) / position_at_end_of_year)

    # Display information
    old_asset_value = round(Decimal(old_price * position_at_end_of_year), 2)
    new_asset_value = round(Decimal(new_price * position_at_end_of_year), 2)

    result = {
        "old_price": old_price,
        "new_price": new_price,
        "old_asset_value": old_asset_value,
        "new_asset_value": new_asset_value,
        "old_nav": nav_at_end_of_year,
        "target_nav": target_nav
    }

    print(f"Old Price: {result['old_price']}, New Price: {result['new_price']}")
    print(f"Old Asset Value: {result['old_asset_value']}, New Asset Value: {result['new_asset_value']}")
    print(f"Old NAV: {result['old_nav']}, Target NAV: {result['target_nav']}")

    # Ask for confirmation
    confirm = input("Do you want to update the price? (yes/no): ")

    if confirm.lower() == 'yes':
        # Update the price
        price_instance, created = Prices.objects.get_or_create(
            security=asset, date=end_of_year_date,
            defaults={'price': new_price}
        )
        if not created:
            price_instance.price = new_price
            price_instance.save()

        result["status"] = "Price updated successfully."
        print("New price is:", price_instance.price)
    else:
        result["status"] = "Price update canceled."

    return result

user = CustomUser.objects.filter(username='Yaroslav').first()
year = 2013
broker_name = 'UBS Pension'
target_nav = 29444.38
asset_name = 'Emerging Markets Equity Fund'

end_of_year_price_correction(user, year, broker_name, target_nav, asset_name)

Old Price: 0.936630, New Price: 0.9366299527568694238659180266
Old Asset Value: 7983.27, New Asset Value: 7983.27
Old NAV: 29444.380402671861, Target NAV: 29444.38


{'old_price': Decimal('0.936630'),
 'new_price': Decimal('0.9366299527568694238659180266'),
 'old_asset_value': Decimal('7983.27'),
 'new_asset_value': Decimal('7983.27'),
 'old_nav': Decimal('29444.380402671861'),
 'target_nav': Decimal('29444.38'),
 'status': 'Price update canceled.'}

In [15]:
from datetime import timedelta

import pandas as pd
from common.models import AnnualPerformance
from utils import broker_group_to_ids, get_fx_rate


def calculate_performance(user, start_date, end_date, selected_brokers_ids, currency_target, is_restricted=None):
    performance_data = {name: Decimal(0) for name in [
        "bop_nav", "invested", "cash_out", "price_change", "capital_distribution",
        "commission", "tax", "fx", "eop_nav", "tsr"
    ]}

    brokers = Brokers.objects.filter(id__in=selected_brokers_ids, investor=user).all()

    for broker in brokers:
        # Calculate BOP NAV
        bop_nav = AnnualPerformance.objects.filter(
            investor=user, broker=broker, year=start_date.year - 1, currency=currency_target
        ).values('eop_nav').first()
        
        if not bop_nav:
            bop_nav = NAV_at_date(user.id, [broker.id], start_date - timedelta(days=1), currency_target)['Total NAV']
        else:
            bop_nav = bop_nav['eop_nav']
        
        performance_data['bop_nav'] += bop_nav

        # Filter transactions
        transactions = Transactions.objects.filter(
            investor=user,
            broker_id=broker.id,
            date__gte=start_date,
            date__lte=end_date,
        )

        if is_restricted is not None:
            transactions = (transactions.filter(security__isnull=False, security__restricted=is_restricted) |
                            transactions.filter(security__isnull=True))

        transactions_df = pd.DataFrame(list(transactions.values('cash_flow', 'type', 'commission', 'date', 'currency')))

        if not transactions_df.empty:
            transactions_df['fx_rate'] = transactions_df.apply(
                lambda row: get_fx_rate(row['currency'], currency_target, row['date']), axis=1
            )

            # Calculate transaction-based metrics
            performance_data['invested'] += round((transactions_df[transactions_df['type'] == 'Cash in']['cash_flow'] * 
                                                   transactions_df[transactions_df['type'] == 'Cash in']['fx_rate']).sum(), 2)
            performance_data['cash_out'] += round((transactions_df[transactions_df['type'] == 'Cash out']['cash_flow'] * 
                                                   transactions_df[transactions_df['type'] == 'Cash out']['fx_rate']).sum(), 2)
            performance_data['commission'] += round((transactions_df['commission'] * transactions_df['fx_rate']).sum(), 2)
            performance_data['tax'] += round((transactions_df[transactions_df['type'] == 'Tax']['cash_flow'] * 
                                              transactions_df[transactions_df['type'] == 'Tax']['fx_rate']).sum(), 2)

        # Calculate asset-based metrics
        assets = Assets.objects.filter(investor=user, brokers=broker)
        if is_restricted is not None:
            assets = assets.filter(restricted=is_restricted)

        for asset in assets:
            asset_realized_gl = asset.realized_gain_loss(end_date, currency_target, broker_id_list=[broker.id], start_date=start_date)
            performance_data['price_change'] += asset_realized_gl["all_time"] if asset_realized_gl else 0
            performance_data['price_change'] += asset.unrealized_gain_loss(end_date, currency_target, broker_id_list=[broker.id], start_date=start_date)
            performance_data['capital_distribution'] += asset.get_capital_distribution(end_date, currency_target, broker_id_list=[broker.id], start_date=start_date)

        # Calculate EOP NAV
        eop_nav = NAV_at_date(user.id, [broker.id], end_date, currency_target)['Total NAV']
        performance_data['eop_nav'] += eop_nav

    # Calculate FX impact
    components_sum = sum(performance_data[key] for key in ['bop_nav', 'invested', 'cash_out', 'price_change', 'capital_distribution', 'commission', 'tax'])
    print("eop_nav", performance_data['eop_nav'], components_sum)
    performance_data['fx'] += performance_data['eop_nav'] - components_sum

    # Calculate TSR
    performance_data['tsr'] = Irr(user.id, end_date, currency_target, broker_id_list=selected_brokers_ids, start_date=start_date)

    # Adjust FX for rounding errors
    performance_data['fx'] = Decimal(0) if abs(performance_data['fx']) < 0.1 else performance_data['fx']

    return performance_data

investor = CustomUser.objects.get(id=2)
start_date = date(2010, 1, 1)
end_date = date(2010, 12, 31)
selected_brokers_ids = broker_group_to_ids("All brokers", investor)

perf = calculate_performance(investor, start_date, end_date, selected_brokers_ids, currency_target='GBP', is_restricted=None)
perf

eop_nav 15039.440342064474 15039.43


{'bop_nav': Decimal('5175.48'),
 'invested': Decimal('8015.30'),
 'cash_out': Decimal('0'),
 'price_change': Decimal('1848.65'),
 'capital_distribution': Decimal('0'),
 'commission': Decimal('0'),
 'tax': Decimal('0'),
 'fx': Decimal('0'),
 'eop_nav': Decimal('15039.440342064474'),
 'tsr': Decimal('0.1927999999999999991562305012848810292780399322509765625')}